In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
import shutil

In [3]:
from subprocess import Popen, PIPE
import time
from IPython.display import clear_output

In [4]:
PATH = "/home/gridsan/ptheron/MoEBERT-fork"
PYPATH = "/home/gridsan/ptheron/.conda/envs/MoEBERT/bin/python"
LOG_PATH = f"{PATH}/logs"

In [5]:
LOG_PATH

'/home/gridsan/ptheron/MoEBERT-fork/logs'

# Download glue datasets and necessary module from huggingface

In [ ]:
!bash ../fetching/sh_dl_ds.sh

# Submit Jobs

In [ ]:
datasets = ['rte', 'cola','mrpc', 'sst2', 'qqp', 'mnli', 'qnli']


### BERT Task specific finetuning

In [ ]:
OUTPUT_DIR = "/home/gridsan/ptheron/MoEBERT-fork/results"
torun = ['rte', 'cola', 'mrpc', 'sst2']
submitted = []

In [ ]:
exit_code = 1
for i,all_id in enumerate(torun):
    if i % 100 == 0:
        clear_output(wait=True)
    print(i)
    while True:
        process = Popen(["sbatch", "submit_first_finetuning.sh", all_id, OUTPUT_DIR], stdout=PIPE)
        (output, err) = process.communicate()
        exit_code = process.wait()
        print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())),output,err)
        if exit_code == 0:
            print(all_id,"submitted!")
            tmp_id = str(output)[-11:-3]
            print("job id:", tmp_id)
            submitted.append(tmp_id)
            break
        time.sleep(50000)
        

### MoEBERT Seeds finetuning

In [94]:
torun = datasets.copy()

30


In [ ]:
submitted = []

In [ ]:

OUTPUT_DIR = "/home/gridsan/ptheron/MoEBERT-fork/results"

In [95]:
exit_code = 1
for i,all_id in enumerate(torun):
    if i % 100 == 0:
        clear_output(wait=True)
    print(i)
    while True:
        process = Popen(["sbatch", "submit_seed.sh", all_id, OUTPUT_DIR], stdout=PIPE)
        (output, err) = process.communicate()
        exit_code = process.wait()
        print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())),output,err)
        if exit_code == 0:
            print(all_id,"submitted!")
            tmp_id = str(output)[-11:-3]
            print("job id:", tmp_id)
            submitted.append(tmp_id)
            break
        time.sleep(50000)
        

0
2023-01-06 11:27:09 b'Submitted batch job 21011683\n' None
../mixture-of-digits_bashes/v2/instance_specific_soft_k_trees_ensemble_learn_permuted_gate_seed61.sh submitted!
job id: 21011683
1
2023-01-06 11:27:09 b'Submitted batch job 21011684\n' None
../mixture-of-digits_bashes/v2/instance_specific_soft_k_trees_ensemble_learn_permuted_gate_seed62.sh submitted!
job id: 21011684
2
2023-01-06 11:27:09 b'Submitted batch job 21011685\n' None
../mixture-of-digits_bashes/v2/instance_specific_soft_k_trees_ensemble_learn_permuted_gate_seed63.sh submitted!
job id: 21011685
3
2023-01-06 11:27:09 b'Submitted batch job 21011686\n' None
../mixture-of-digits_bashes/v2/instance_specific_soft_k_trees_ensemble_learn_permuted_gate_seed64.sh submitted!
job id: 21011686
4
2023-01-06 11:27:09 b'Submitted batch job 21011687\n' None
../mixture-of-digits_bashes/v2/instance_specific_soft_k_trees_ensemble_learn_permuted_gate_seed65.sh submitted!
job id: 21011687
5
2023-01-06 11:27:09 b'Submitted batch job 210116

# Cancel jobs

In [625]:
for job in submitted:
    process = Popen(['scancel',job], stdout=PIPE)
    (output, err) = process.communicate()
    exit_code = process.wait()
    if exit_code ==0:
        print(job, "deleted!")

20989143 deleted!
20989144 deleted!
20989145 deleted!
20989146 deleted!
20989147 deleted!
20989148 deleted!
20989149 deleted!
20989150 deleted!
20989151 deleted!
20989152 deleted!
20989153 deleted!
20989154 deleted!
20989155 deleted!
20989156 deleted!
20989157 deleted!
20989158 deleted!
20989159 deleted!
20989160 deleted!
20989161 deleted!
20989162 deleted!
20989163 deleted!
20989164 deleted!
20989165 deleted!
20989166 deleted!
20989167 deleted!
20989168 deleted!
20989169 deleted!
20989170 deleted!
20989171 deleted!
20989172 deleted!
20989173 deleted!
20989174 deleted!
20989175 deleted!
20989176 deleted!
20989177 deleted!
20989178 deleted!
20989179 deleted!
20989180 deleted!
20989181 deleted!
20989182 deleted!
20989183 deleted!
20989184 deleted!
20989185 deleted!
20989186 deleted!
20989187 deleted!
20989188 deleted!
20989189 deleted!
20989190 deleted!
20989191 deleted!
20989192 deleted!
20989193 deleted!
20989194 deleted!
20989195 deleted!
20989196 deleted!
20989197 deleted!
20989198 d